In [2]:
%load_ext autoreload
%autoreload 2

**NOTE - This is a jupyter notebook running on a python kernel. You can run multiline bash commands in a cell with %%bash, or run single bash commands by prepending the command with "!". If running externally in a bash terminal, the mentioned additions are not needed**

**If you are using EMPIAR10164 tutorail dataset, the following two cells can be skipped**

**If you have your own TOMO5 collected data, begin your data processing by copying the mdoc files from the raw data directory to the working mdoc directory.**

*The below script will copy the mdoc files while renaming outputted TOMO5 mdoc files with a better regular expression system (from Position_X_X (the default TOMO5 naming convention) to TS_XXX) while creating a "mapping" text file that tracks the new and old names.* (X = a digit)
<br>*The reason for this is that Warp assembles outputted tiltstacks with the corresponding names of the mdoc files it is given. This is all that's needed to rename the tilstack*
<br>Changing the regular expression system will allow much easier interfacing with external processing suites (i.e. gapstop)
<br>If your naming convention is different from Position_X_X, you will have to modify the script

In [ ]:
%%bash
# Script to copy and rename MDOC files to match renamed tilt series files
# Usage: ./copy_mdoc_files.sh [-s source_directory] [-t target_directory] [-n start_number] [-f file_list] [-m mapping_file] [-d]

set -e

# SET THESE VARIABLES

START_NUMBER=1 #What to start the TS_XXX.mrc numbering from (XXX = digits. starting from "1" will begin numbering with "001")
DRY_RUN=false #test what the outputs will be before running for real (options = true,false)
FILE_LIST="" #path to a text file containing a list of all the mdocs being copied (text file must have one file per line) - not necessary
MAPPING_FILE="/path/to/mapping_file.txt" #mapping file = text file containing a list of the original files and the corresponding copied name
SRC_DIR="rawdata/mdoc" #source directory
TARGET_DIR="mdocs" #target director

# Required arguments check
if [ -z "$SRC_DIR" ] || [ -z "$TARGET_DIR" ]; then
    echo "Error: Both source (-s) and target (-t) directories are required." >&2
    usage
fi

# Cleanup paths
SRC_DIR="${SRC_DIR%/}"
TARGET_DIR="${TARGET_DIR%/}"

# Set default mapping file if not provided
if [ -z "$MAPPING_FILE" ]; then
    MAPPING_FILE="$TARGET_DIR/mdoc_copy_mapping.txt"
fi

echo "Source directory: $SRC_DIR"
echo "Target directory: $TARGET_DIR"
echo "Starting number: $START_NUMBER"
echo "File list: ${FILE_LIST:-[all *.mdoc]}"
echo "Mapping file: $MAPPING_FILE"
echo "Dry run: $DRY_RUN"

# Prepare target and mapping file if not a dry run
if [ "$DRY_RUN" = false ]; then
    mkdir -p "$TARGET_DIR"
    echo "Created target directory: $TARGET_DIR"

    mkdir -p "$(dirname "$MAPPING_FILE")"
    > "$MAPPING_FILE"
    echo "Initialized mapping file: $MAPPING_FILE"
fi

# Collect and parse filenames
declare -a file_info
process_files() {
    local file="$1"
    local name=$(basename "$file")

    if [[ "$name" =~ ^([A-Za-z_]+)([0-9]+)(_([0-9]+))?\.mdoc$ ]]; then
        prefix="${BASH_REMATCH[1]}"
        main="${BASH_REMATCH[2]}"
        suffix="${BASH_REMATCH[4]:-0}"
        file_info+=("$prefix:$(printf "%08d" $main):$(printf "%08d" $suffix):$file")
    else
        file_info+=("ZZZZ:99999999:99999999:$file")
    fi
}

# Load file list
if [ -n "$FILE_LIST" ]; then
    while IFS= read -r rel_path || [[ -n "$rel_path" ]]; do
        [[ -z "$rel_path" ]] && continue
        abs_path="$SRC_DIR/$rel_path"
        [ -f "$abs_path" ] && process_files "$abs_path" || echo "Warning: File not found: $abs_path"
    done < "$FILE_LIST"
else
    for file in "$SRC_DIR"/*.mdoc; do
        [ -f "$file" ] && process_files "$file"
    done
fi

# Sort
IFS=$'\n' sorted_files=($(sort -t ':' -k1,1 -k2,2n -k3,3n <<<"${file_info[*]}"))
unset IFS

# Copy and map
new_index=$START_NUMBER
for entry in "${sorted_files[@]}"; do
    IFS=':' read -r _ _ _ src_file <<< "$entry"
    base=$(basename "$src_file")
    new_name="TS_$(printf "%03d" $new_index).mdoc"
    new_path="$TARGET_DIR/$new_name"

    echo "Copying: $base -> $new_name"
    if [ "$DRY_RUN" = false ]; then
        cp "$src_file" "$new_path"

        # Record both MDOC and corresponding MRC mapping
        orig_mrc="${base/.mdoc/.mrc}"
        new_mrc="${new_name/.mdoc/.mrc}"
        echo "$orig_mrc:$new_mrc" >> "$MAPPING_FILE"
    fi

    new_index=$((new_index + 1))
done

echo "Processed $((new_index - START_NUMBER)) files"
[ "$DRY_RUN" = false ] && echo "Mapping written to: $MAPPING_FILE"
echo "Done!"

**Now that we have the renamed mdoc files, it's time to symlink the other raw frames into an input directory**

In [ ]:
!ln -s $PWD/rawdata/frames/*.mrc frames

**Set some python variables that carried for downstream processing**
- ts_settings = name of settings file that warp will use for tilt-series preprocessing steps
- fs_settings = name of settings file that warp will use for frame-series preprocessing steps
- bin_1_angpix = the pixel size of your raw data
- bin_fact = the factor you will want your final tomogram binned by

In [ ]:
ts_settings = 'warp_tiltseries.settings'
fs_settings = 'warp_frameseries.settings'
bin_1_angpix = #your bin1 value
bin_fact = #your final binning factor

final_bin_pix = bin_1_angpix * bin_fact

!echo {ts_settings}
!echo {fs_settings}
!echo {bin_1_angpix}
!echo {final_bin_pix}

**CREATE SETTINGS FILE FOR WARP**

*Explanations of commands*
- create settings creates a corresponding .settings file (whose name you chose previously) which defines a set of parameters for warp to run with. (i.e. defined inputs outputs)
- folder_data = raw data folder (for corresponding settings file)
- folder_processing = working directory which output files will be saved to (and often read from)
- output = name of .settings file
- extension = what kind of files should be looked for in the raw data directory (.tif, .mrc, .eer)
- angpix = pixel size of the raw data (previously set)
- gain_path = path to your gain reference if you are gain correcting .tif and .eer files (recommended)
- gain_flip = gain often has to be flipped or rotated to align with your data (the facility should tell you) - can be defined as x or y
- exposure = per tilt exposure or per frame exposure (defined by - sign) in angstroms/pixel^2
- tomo_dimensions = set to the camera size.

*eer specific commands below:*
- eer_ngroups = how many pseudoframes you should align to. (Ideally you should have between 0.5-1.5 exposure/pseudoframe (source RELION documentation))

**IF YOU NEED MORE INFO ON COMMANDS CHECK THE WARP API WEBSITE**
**IF YOU NEED MORE INFO ON THE FILE PARAMETERS, CHECK WITH IMOD COMMAND "header /path/to/file/" to load imod run - "module load cryo contribs imod" in terminal**

In [ ]:
%%bash -s "$ts_settings" "$fs_settings" "$bin_1_angpix"
#bringing the previously set python variables into bash as 1 to and 3

###DEFAULT VALUES FOR K3 ONLY - COMMENT OUT IF NOT USING K3

WarpTools create_settings \
--folder_data frames \
--folder_processing warp_frameseries \
--output "$2" \
--extension "*.mrc" \
--angpix "$3" \
--exposure ##set your exposure/tilt here

WarpTools create_settings \
--output "$1" \
--folder_processing warp_tiltseries \
--folder_data tomostar \
--extension "*.tomostar" \
--angpix "$3" \
--tomo_dimensions 5760x4092x2000

###THIS IS EER ONLY - COMMENT OUT IF NOT USING FALCON4i

#WarpTools create_settings \
#--folder_data frames \
#--folder_processing warp_frameseries \
#--output {fs_settings} \
#--extension "*.eer" \
#--angpix {bin_1_angpix} \
#--gain_path gain.mrc \
#--eer_ngroups 9 \
#--gain_flip_y \
#--exposure 2.69

##WarpTools create_settings \
#--output {ts_settings} \
#--folder_processing warp_tiltseries \
#--folder_data tomostar \
#--extension "*.tomostar" \
#--angpix {bin_1_angpix} \
#--gain_path gain.mrc \
#--gain_flip_y \
#--eer_ngroups 9 \
#--exposure 2.69 \
#--tomo_dimensions 4096x4096x2000

*note - you may encounter bad tilts or bad frames that you will want to deselect. when discovered, use the following command*

<br> WarpTools change_selection --settings 'corresponding_settings_file.settings' --input_data 'text_file_containing_one_bad_data_per_line.txt' --deselect
- when placing the bad data into the text file, make sure that the you replace the processing directory (e.g. warp_frameseries) with the raw directory (e.g. frames). Not sure why this is, but it just is.
- selecting specific frames for processing is done similarly, just use the --select flag instead of --deselect

**Now that you have created the settings files, try to run motion correction with warp with the following command**
<br> *if your settings file is not pointing in the correct direction, warp will let you know by stating "0 files found"*

In [ ]:
!WarpTools fs_motion_and_ctf \
--settings {fs_settings} \
--m_grid 1x1x3 \
--c_grid 2x2x1 \
--c_range_max 7 \
--c_defocus_max 8 \
--c_use_sum \
--out_averages \
--out_average_halves

Warp can estimate metrics to empirically determine the quality of your frames. These metrics are:
- Between frame motion in the first 1/3 of collected frames of each frameseries (in Angstrom)
- Defocus of each Frameseries (in micrometers)
- Astigmatism of each frameseries (in micrometers)
- CTF resolution of each frameseries (in Angstrom)
- Phase shift (if using a VPP (we don't have))

<br> *note - if the frames are pre-gain corrected before input into warp, I don't think these metrics are accurate*

**Try outputting a histogram of warp's estimated ctf and motion metrics using the following command**



In [ ]:
!WarpTools filter_quality --settings {fs_settings} --histograms

**Now that we have a histogram representation of the per-frame metrics for CTF, Motion, and more, we can set a cutoff criteria where frames within are selected for downstream processing**

<br> *here is the command with flags of interest:*

<br> WarpTools filter_quality --defocus "min" "max" --astigmatism "min" "max" --phase "min" "max" --resolution "min" "max" --motion "min" "max"

<br> *note that the astigmatism min and max values are standard deviations from the dataset mean*
<br> *you can also see which frames actually meet the cutoff criteria with -o /path/to/text/.txt

**Now, give warp your mdoc files**

*Warp reads the mdoc files and outputs a "tomostar" file. This is the internal format it uses to keep track of what frameseries contribute to which tiltstack. It also contains other metrics.*

Example:

data_

<br> loop_
<br> _wrpMovieName #1
<br> _wrpAngleTilt #2
<br> _wrpAxisAngle #3
<br> _wrpDose #4
<br> _wrpAverageIntensity #5
<br> _wrpMaskedFraction #6
<br>   ../warp_frameseries/Position_3_040_-54.00_20250723_134016_fractions.mrc  -54.00  97.780       124.8   547.000  0.000
<br>   ../warp_frameseries/Position_3_039_-51.00_20250723_133924_fractions.mrc  -51.00  97.780       121.6   597.000  0.000
<br>   ../warp_frameseries/Position_3_036_-48.00_20250723_133623_fractions.mrc  -48.00  97.780         112   648.500  0.000
<br>   ../warp_frameseries/Position_3_035_-45.00_20250723_133531_fractions.mrc  -45.00  97.780       108.8   695.000  0.000
<br>   ../warp_frameseries/Position_3_032_-42.00_20250723_133228_fractions.mrc  -42.00  97.780   99.200005   737.000  0.000
<br>   ../warp_frameseries/Position_3_031_-39.00_20250723_133136_fractions.mrc  -39.00  97.780          96   782.500  0.000
<br>   ../warp_frameseries/Position_3_028_-36.00_20250723_132837_fractions.mrc  -36.00  97.780        86.4   822.500  0.000
<br>   ../warp_frameseries/Position_3_027_-33.00_20250723_132745_fractions.mrc  -33.00  97.780   83.200005   851.000  0.000
<br>   ../warp_frameseries/Position_3_024_-30.00_20250723_132447_fractions.mrc  -30.00  97.780        73.6   880.500  0.000

In [ ]:
##NOTE - if acquisition was done on tomo5, the tilt axis will not be accurate. You will have to manually calculate it (+-90 from the given value in the mdoc). The handedness should be given by the facility.
#if serialEM/PACE-Tomo was used to acquire tiltseries, the override axis flag is likely not necessary

!WarpTools ts_import \
--mdocs mdocs \
--frameseries warp_frameseries \
--tilt_exposure 3.2 \
--min_intensity 0.3 \
--dont_invert \
--override_axis 'your_tilt_axis_here' \
--output tomostar

**Now, assemble your tiltstack from the tomostar data**

In [ ]:
!WarpTools ts_stack \
--settings {ts_settings} \
--angpix {final_bin_pix}

At this stage, you will want to copy the tiltstack and the tomostar file onto a local computer and parse through the tilts one by one. Remove any bad tilts from the tomostar file manually and delete the .xml files from the warp_tiltseries directory. 

<br> Sync the modified tomostar files back to the tomostar directory and reassemble the tiltstack. Bad tilts will have been removed this way, and your alignment will be much smoother.


**Once you are satisfied with the included tilts, pass the tiltstack and the rawtlt file to AreTomo for alignment. If desired, IMOD alignment must be done manually, and is unable to be performed in this notebook.**
<br> This script will also pass the AreTomo alignments back to Warp via WarpTools ts_import_alignments

Note - you will have to edit the script to give AreTomo the tiltaxis value. (the second "1" value is fine)

In [ ]:
%%bash

#the cuda libraries are found in the conda environment. You must export the libraries from the conda environment to the LD_LIBRARY_PATH which is how aretomo will find the cuda libraries
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH

##run aretomo on the tiltstack externally to generate tiltseries alignment

# Function to get available GPUs (not currently running CUDA processes)
get_available_gpus() {
    local available_gpus=()
    
    # Check if nvidia-smi is available
    if ! command -v nvidia-smi &> /dev/null; then
        echo "Warning: nvidia-smi not found. Cannot detect GPU usage."
        return 1
    fi
    
    # Get list of all GPU IDs
    local all_gpus=($(nvidia-smi --list-gpus | grep -oP 'GPU \K[0-9]+'))
    
    if [ ${#all_gpus[@]} -eq 0 ]; then
        echo "Warning: No GPUs detected."
        return 1
    fi
    
    echo "Detected GPUs: ${all_gpus[*]}"
    
    # Check each GPU for running processes
    for gpu_id in "${all_gpus[@]}"; do
        # Get processes running on this GPU
        local processes=$(nvidia-smi -i "$gpu_id" --query-compute-apps=pid --format=csv,noheader,nounits 2>/dev/null)
        
        if [ -z "$processes" ] || [ "$processes" = "No running processes found" ]; then
            available_gpus+=("$gpu_id")
            echo "GPU $gpu_id: Available"
        else
            echo "GPU $gpu_id: In use (PIDs: $(echo $processes | tr '\n' ' '))"
        fi
    done
    
    if [ ${#available_gpus[@]} -eq 0 ]; then
        echo "Warning: No available GPUs found. All GPUs are currently in use."
        return 1
    fi
    
    echo "Available GPUs: ${available_gpus[*]}"
    printf '%s\n' "${available_gpus[@]}"
    return 0
}

# Function to wait for a GPU to become available
wait_for_available_gpu() {
    local max_wait_time=300  # Maximum wait time in seconds (5 minutes)
    local wait_interval=10   # Check interval in seconds
    local elapsed_time=0
    
    while [ $elapsed_time -lt $max_wait_time ]; do
        local available_gpus_output
        available_gpus_output=$(get_available_gpus 2>/dev/null)
        local gpu_check_result=$?
        
        if [ $gpu_check_result -eq 0 ] && [ -n "$available_gpus_output" ]; then
            # Extract just the GPU IDs (last line of output should contain the available GPUs)
            local available_gpu=$(echo "$available_gpus_output" | tail -n 1)
            echo "$available_gpu"
            return 0
        fi
        
        echo "No GPUs available. Waiting ${wait_interval}s... (${elapsed_time}/${max_wait_time}s elapsed)"
        sleep $wait_interval
        elapsed_time=$((elapsed_time + wait_interval))
    done
    
    echo "Timeout: No GPU became available within ${max_wait_time} seconds"
    return 1
}

# Function to run AreTomo on a specific GPU
run_aretomo_on_gpu() {
    local gpu_id=$1
    local tiltstack=$2
    local dir=$3
    local base=$4
    
    echo "Running AreTomo on GPU $gpu_id for $tiltstack"
    
    # Set CUDA device - AreTomo will see this as GPU 0
    export CUDA_VISIBLE_DEVICES=$gpu_id
    
    ##EDIT TILTAXIS HERE
    $AreTomo -InMrc "$tiltstack" \
    -OutMrc "$dir/${base}_aligned.mrc" \
    -AngFile "${tiltstack%.st}.rawtlt" \
    -TiltAxis $$$ 1 \
    -OutImod 1 \
    -AlignZ 253 \
    -VolZ 0 \
    -DarkTol 0 \
    -Patch 6 6
    
    local result=$?
    
    # Unset CUDA device restriction
    unset CUDA_VISIBLE_DEVICES
    
    return $result
}

echo "Here we go..."

echo "Looking for files in: warp_tiltseries/tiltstack/*/*.st"

# Check if any .st files exist
if ls warp_tiltseries/tiltstack/*/*.st 1> /dev/null 2>&1; then
    echo "Found .st files:"
    ls warp_tiltseries/tiltstack/*/*.st
else
    echo "No .st files found matching pattern: warp_tiltseries/tiltstack/*/*.st"
    echo "Directory contents:"
    ls -la "warp_tiltseries"
    exit 1
fi

# Check if aretomo_alignments directory exists, create if it doesn't
if [ ! -d "warp_tiltseries/aretomo_alignments" ]; then
    echo "Creating aretomo_alignments directory..."
    mkdir -p warp_tiltseries/aretomo_alignments
else
    echo "aretomo_alignments directory already exists"
fi

# Get initial list of available GPUs
echo "Checking GPU availability..."
available_gpus_output=$(get_available_gpus)
gpu_check_result=$?

if [ $gpu_check_result -ne 0 ]; then
    echo "Error: Cannot proceed without GPU detection. Falling back to default behavior."
    # You could set a fallback here, or exit
    available_gpus=()
else
    # Parse available GPUs from output
    mapfile -t available_gpus < <(echo "$available_gpus_output" | tail -n +1 | grep -E '^[0-9]+$')
fi

# Track successful moves
successful_moves=0
total_positions=0
failed_positions=()
current_gpu_index=0

for tiltstack in warp_tiltseries/tiltstack/*/*.st; do
    # Get the directory and base filename
    dir=$(dirname "$tiltstack")
    base=$(basename "$tiltstack" .st)
    
    # Extract position name from directory path (e.g., Position_6, Position_7_2)
    position_name=$(basename "$dir")
    
    echo "Processing: $tiltstack"
    echo "Position: $position_name"
    
    total_positions=$((total_positions + 1))
    
    # Determine which GPU to use
    if [ ${#available_gpus[@]} -gt 0 ]; then
        # Use round-robin assignment of available GPUs
        gpu_id=${available_gpus[$current_gpu_index]}
        current_gpu_index=$(( (current_gpu_index + 1) % ${#available_gpus[@]} ))
        
        echo "Assigned to GPU: $gpu_id"
        run_aretomo_on_gpu "$gpu_id" "$tiltstack" "$dir" "$base"
        aretomo_result=$?
    else
        # No specific GPU available, wait for one or fall back
        echo "No GPUs initially available. Waiting for an available GPU..."
        gpu_id=$(wait_for_available_gpu)
        wait_result=$?
        
        if [ $wait_result -eq 0 ] && [ -n "$gpu_id" ]; then
            echo "GPU $gpu_id became available"
            run_aretomo_on_gpu "$gpu_id" "$tiltstack" "$dir" "$base"
            aretomo_result=$?
        else
            echo "Running AreTomo without specific GPU assignment..."
            #replace XXX with the correct tiltaxis value
            $AreTomo -InMrc "$tiltstack" \
            -OutMrc "$dir/${base}_aligned.mrc" \
            -AngFile "${tiltstack%.st}.rawtlt" \
            -TiltAxis XXX 1 \
            -OutImod 1 \
            -AlignZ 253 \
            -VolZ 0 \
            -DarkTol 0 \
            -Patch 6 6
            aretomo_result=$?
        fi
    fi
    
    # Check if AreTomo succeeded before moving directories
    if [ $aretomo_result -eq 0 ]; then
        echo "AreTomo processing successful for $position_name"
        
        # Look for the generated Imod directory
        imod_dir="$dir/${base}_aligned_Imod"
        if [ -d "$imod_dir" ]; then
            echo "Found Imod directory: $imod_dir"
            
            # Create position subdirectory in aretomo_alignments
            target_dir="warp_tiltseries/aretomo_alignments/${position_name}_Imod"
            
            echo "Moving to: $target_dir"
            
            # Create target directory if it doesn't exist
            mkdir -p "$target_dir"
            
            # Move all files from Imod directory to target, renaming files to remove "_aligned"
            move_success=true
            failed_files=()
            for file in "$imod_dir"/*; do
                if [ -f "$file" ]; then
                    filename=$(basename "$file")
                    
                    # Check if file matches Position_*_aligned pattern (handles Position_#_aligned and Position_#_#_aligned)
                    if [[ "$filename" =~ Position_[0-9]+(_[0-9]+)*_aligned ]]; then
                        # Remove "_aligned" from filename completely
                        new_filename=$(echo "$filename" | sed 's/_aligned//')
                        echo "  Renaming: $filename -> $new_filename"
                        if mv "$file" "$target_dir/$new_filename"; then
                            :  # Success, do nothing
                        else
                            echo "  ERROR: Failed to move $filename"
                            failed_files+=("$filename")
                            move_success=false
                        fi
                    else
                        # File doesn't match pattern, move as-is
                        echo "  Moving: $filename"
                        if mv "$file" "$target_dir/$filename"; then
                            :  # Success, do nothing
                        else
                            echo "  ERROR: Failed to move $filename"
                            failed_files+=("$filename")
                            move_success=false
                        fi
                    fi
                fi
            done
            
            # Remove empty original Imod directory if all moves were successful
            if [ "$move_success" = true ]; then
                rmdir "$imod_dir"
                echo "Removed original Imod directory: $imod_dir"
                successful_moves=$((successful_moves + 1))
            else
                echo "Some files failed to move for $position_name:"
                printf "  - %s\n" "${failed_files[@]}"
                failed_positions+=("$position_name")
            fi
        else
            echo "Warning: Expected Imod directory not found: $imod_dir"
            failed_positions+=("$position_name (Imod directory not found)")
        fi
    else
        echo "AreTomo processing failed for $position_name"
        failed_positions+=("$position_name (AreTomo failed)")
    fi
    
    echo "---"
done

echo "Processing complete: $successful_moves out of $total_positions positions successfully moved"

# Report failed positions if any
if [ ${#failed_positions[@]} -gt 0 ]; then
    echo "Failed positions:"
    printf "  - %s\n" "${failed_positions[@]}"
fi

# Run WarpTools if all positions were successfully moved
if [ $successful_moves -eq $total_positions ] && [ $total_positions -gt 0 ]; then
    echo "All Imod directories were moved and renamed successfully!"
    echo "Running WarpTools ts_import_alignments..."
    
    WarpTools ts_import_alignments \
    --settings warp_tiltseries.settings \
    --alignments warp_tiltseries/aretomo_alignments \
    --alignment_angpix 1.59
    
    if [ $? -eq 0 ]; then
        echo "WarpTools ts_import_alignments completed successfully!"
    else
        echo "WarpTools ts_import_alignments failed!"
    fi
else
    echo "Not all positions were successfully processed. Skipping WarpTools import."
    echo "Successfully moved: $successful_moves"
    echo "Total positions: $total_positions"
fi

if [ $successful_moves -eq $total_positions ] && [ $total_positions -gt 0 ]
then
    echo "Success, looks good!"
    echo "Imod directories have been moved to warp_tiltseries/aretomo_alignments/"
else
    echo "Finished with errors :("
fi

**Now that you have the aligned tiltstack, we can estimate the defocus handedness of the tomogram to make sure we get the right orientation**

In [ ]:
!WarpTools ts_defocus_hand \
--settings {ts_settings} \
--check

**If necessary, flip defocus with the following script** (uncomment it)

In [ ]:
#!WarpTools ts_defocus_hand \
#--settings {ts_settings} \
#--set_flip

**Once we get the correct handedness, estimate the ctf gradient for the 3d volume, not just per tilt. This will be used to fourier reconstruct the tomograms.**

In [ ]:
!WarpTools ts_ctf \
--settings {ts_settings} \
--range_high 7 \
--defocus_max 8

**You should now be all set to reconstruct the tomograms**

In [4]:
#!WarpTools ts_reconstruct \
#--settings {ts_settings} \
#--halfmap_frames \
#--angpix {final_bin_pix}

#the following command will copy all the generated tomograms and pngs into first_pass directory so you can easily copy it locally with rsync and visualize with IMOD

!mkdir first_pass_ts
import subprocess
subprocess.run(f"cp $PWD/warp_tiltseries/reconstruction/*_{final_bin_pix}Apx* first_pass_ts", shell=True)

**The following is an extra but helpful step to output a medial slice snapshot of each tomogram for visualization**

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display, Image

from glob import glob

import PIL
images = [ PIL.Image.open(f) for f in glob('warp_tiltseries/reconstruction/*.png') ]
def img2array(im):
    if im.mode != 'L':
        im = im.convert(mode='L')
    return np.frombuffer(im.tobytes(), dtype='uint8').reshape((im.size[1], im.size[0]))
np_images = [ img2array(im) for im in images ]

plt.figure(figsize=(20,20))
columns = 8
rows = len(np_images) // columns + (1 if len(np_images) % columns != 0 else 0)

for i, img in enumerate(np_images):
    plt.subplot(rows, columns, i + 1)
    plt.imshow(img, cmap='gray_r')
    plt.axis('off')

plt.subplots_adjust(wspace=0.01, hspace=0.01)  # Remove spacing
plt.show()
plt.close()

**If you want to take your tomograms and process them downstream in GAPSTOP for 3D template matching, run this below cell. Otherwise you're now done with the notebook. You might want to iteratively process until you have a well-aligned and reconstructed tomogram you are happy with.** 
**Just edit a couple variables at the top of the notebook and re-run.**

In [4]:
%%bash

mkdir -p gapstop/inputs/tlt gapstop/inputs/ctf_dose gapstop/inputs/wedgelist gapstop/inputs/tomograms
ln -s $PWD/warp_tiltseries/reconstruction/*.mrc gapstop/inputs/tomograms
ln -s $PWD/warp_tiltseries/aretomo_alignments/*/*.tlt gapstop/inputs/tlt/
ln -s $PWD/warp_tiltseries/*.xml gapstop/inputs/ctf_dose/